<h1>CP2410 Assignment 1</h1>
<h2>By Ryan Wong</h2>

<h3><u>Introduction:</u></h3>  
The task selected here is option 2, involving:
<br> 1. The plotting of the best path given a large list of coordinates
<br> 2. With a set starting and ending point with coordinates [0.0, 0.0]
<br> 3. And a condition of: if the city index of the 10th stop is not a prime number, it takes 10% longer to reach the next city (e.g if 10th is not prime, 10 to 11 is 10% longer)

<h3><u>First Algorithm - Brute Force</u></h3>
<br> In my first attmept to solve the problem, I attempt to use the simplest method of brute force to find a best solution.
<br> This method very simply attempts to try every possible solution to the problem, saving the best solution found.

It should be noted these are my input data assumptions:
1. First row of data is the column names.
2. Data has columns: 'id','x-coord','y-coord', In this order.
3. Data is initially sorted by id
4. Data has no missing values
5. Data starts at id 0

Firstly, I import some modules to allow me to handle the data:
<br> > csv: To read the data in the csv
<br> > math: To calculate squareroots to find exact distances beteen points
<br> > sys: To change recursion limit as the number of data elements is high
<br>
<br>
Secondly, I initialize some global variables to be used to hold necessary data.
(Variables are explained by their names)

In [ ]:
# Import csv module to interact with the input data
import csv
import math
import sys

points_list = []
shortest_pt_distance = float('inf')
best_path = []
best_distance = float('inf')
input_data = "../input/a1-data/first3.csv"  # Enter filename of input data here.
output_data = "output.csv"  # Enter filename of output data here
starting_point = [0.0, 0.0, 0.0]  # Starting point declared here. Format: [0.0, x-coord, y-coord]
ending_point = [0.0, 0.0, 0.0]  # Ending point declared here. Format: [0.0, x-coord, y-coord]

Using the <b>csv</b> module, I create a reader and copy the data from the csv of input data into the 'points_list', and prepare a function to write the end output into a csv
<br>
<br>Time efficiency: n<sup>2</sup> + 2n + 3

In [ ]:
with open(input_data, 'r') as csvfile:  # reading the csv file
        csvreader = csv.reader(csvfile)     # Creating a reader
        col_names = next(csvreader)         # Extract column names
        for point_STR in csvreader:         # Add data points into internal list
            point_FLT = []                  # Convert the data points from String to Float
            for i in point_STR:
                point_FLT.append(float(i))
            points_list.append(point_FLT)
        
def save_best_path():
    with open(output_data, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(best_path)

Using the <b>sys</b> module, I prepare the recursion limit to accept the large amount of data.
This is because the default limit is 999, but in this algorithm
I plan to use a depth of recursion for each point in the path plotted, so 10% of the full data
would reach a depth of over 11 000

In [ ]:
if sys.getrecursionlimit() < len(points_list): # Checks if recursion limit will be hit
    sys.setrecursionlimit(2*len(points_list))  # Extends it if yes

Using the <b>math</b> module, I create functions to calculate the distance between 2 points, given their indexes.
Additional functions are created to calculate the distance from the starting point to the first and the from the last point to the ending point.
<br>
<br>Time efficiency: 13

In [ ]:
def find_dist(pt1, pt2):                # Function to find the distance between 2 points
    pt1 = int(pt1)                      # Calculation below assumes points are on flat surface
    pt2 = int(pt2)
    distance = math.sqrt(((points_list[pt1][1] - points_list[pt2][1])**2) + ((points_list[pt1][2] - points_list[pt2][2])**2))
    return distance

def find_dist_start(pt):                # Extra function used to account for starting point if provided
    pt = int(pt)                        # Calculation below assumes points are on flat surface
    distance = math.sqrt(((starting_point[1] - points_list[pt][1])**2) + ((starting_point[2] - points_list[pt][2])**2))
    return distance

def find_dist_end(pt):                  # Extra function used to account for ending point if provided
    pt = int(pt)                        # Calculation below assumes points are on flat surface
    distance = math.sqrt(((ending_point[1] - points_list[pt][1])**2) + ((ending_point[2] - points_list[pt][2])**2))
    return distance

Since the process of brute forcing the answer will absolutely be very long, I attempt to shave some time off with the next 3 functions:
<br>
<br><b>min_point_distance:</b> This function checks every point to find their distance to every other point, saving the shortest distance found.
<br><b>path_is_viable:</b> This function uses the shortest distance found by the function above, the currently plotted paths distance, and the number of remaining points to guess a minimum possible distance that path could possibly be.
<br><b>path_is_better:</b> Simply check if the currently plotted path is better, saving the path and distance if yes.
<br>
<br>Time efficiency: 4n<sup>2</sup> + n + 17

In [ ]:
# Recursive function to find shortest point-to-point distance
def min_point_distance():
    global shortest_pt_distance
    num_pts_left = len(points_list)
    points_left = points_list.copy()    # Creates a separate list to be sorted by coordinates
    best_pt1 = 0.0
    best_pt2 = 0.0
    for i in range(num_pts_left - 1):               # Begins a loop to check every point
        for j in range(i+1, num_pts_left):          # Begins second loop to check against other points
            if find_dist(points_left[i][0], points_left[j][0]) < shortest_pt_distance:    # Checks if better
                shortest_pt_distance = find_dist(points_left[i][0], points_left[j][0])    # Saves better difference
                best_pt1 = points_left[i][0]        # Saves point indexes
                best_pt2 = points_left[j][0]
    print("Shortest distance is between point: ", best_pt1, " and ", best_pt2)
    print("Shortest distance is: ", shortest_pt_distance)
    return shortest_pt_distance

def path_is_viable(num_pts_left, curr_distance, curr_pt):
    if (((num_pts_left) * shortest_pt_distance) + curr_distance + find_dist_end(curr_pt)) < best_distance:
        return True
    else:
        return False
    
def path_is_better(curr_path, curr_dist):
    global best_path
    global best_distance
    if curr_dist < best_distance:
        best_path = curr_path.copy()
        best_distance = curr_dist

The following is the main algorithm of this attempt. The idea simply is that every possible path is created and measured for its total distance of the path, with the path information being saved if it is better.
<br>
<br>This is done by creating a recursive function that systematically goes through each possible point, using the 'path_is_viable' function after each plotted node to try to lessen the combinations plotted, by skipping paths where there is no possible way that it mght be better.
<br>
<br>Time complexity: n!

In [ ]:
# Recursive function to plot every possible path
def plot_path(num_pts_left, curr_path, pts_left, pts_list, curr_dist):
    if len(curr_path) == 0 and len(pts_left) == 0 and pts_list != 0:    # Makes a separate list to track unused pts
        pts_left = pts_list.copy()
    if len(curr_path) == 0:
        curr_path.append(starting_point)    # Add starting point
    for point in pts_left:                  # Begins loop for each point in data to plot each path possible
        curr_path.append(point)             # Adds point in path to keep track of points used
        pts_left.remove(point)              # Removes this point from tracking of unused points

        if len(starting_point) == 3 and len(curr_path) == 2:    # Check for starting point
            curr_dist = find_dist_start(point[0])               # Adding distance from start to first point if yes
        else:
            curr_dist += find_dist(curr_path[-2][0], point[0])  # Update distance of current path

        if num_pts_left == 1:                                   # Checks if this was the last point.
            curr_dist += find_dist_end(point[0])                # Adding distance from this point to end if yes.
            path_is_better(curr_path, curr_dist)

        if path_is_viable(num_pts_left, curr_dist, point[0]):
            plot_path(num_pts_left - 1, curr_path, pts_left, pts_list, curr_dist)

        curr_dist -= find_dist(curr_path[-2][0], point[0])
        pts_left.insert(int(point[0]), point)
        pts_left.sort()
        curr_path.remove(point)



min_point_distance()
plot_path(len(points_list), [], [], points_list, 0)
best_path.remove(starting_point)
save_best_path()
print("Best distance is: ", best_distance)
print("Using best path: ", best_path)

While the above test results show good effectiveness with dataset of size 3, due to the extremely low time effeciency of n!, a test run i had attempted with 1900 results(1%) was not remotely complete after half an hour.

<h3><u>Algorithm 1 review</u></h3>
<br>Due to the nature of comparing every possible combination of path, this algorithm should <b>in theory</b> provide the best path if used for any given dataset.
<br>However, due to the extremely poor time efficiency, even a high end computer(of 2019) with a hypothetical processor of 4GHz would theoretically require an amount of time that returns as 'undefined' by calculators(19 000! / 4 billion calculations per second).
<br>Thus, this algorithm could not be used to plot a path more than a few points long without requiring years of processing time to plot.

<h3><u>Second Algorithm - Closest Point</u></h3>
<br>This algorithm uses the simple logic of connecting to the closest point available.

This new algorithm uses many of the same functions and modules as the above, so the next cell of this notebook contains the reused code from before.
<br>Additionally, the assumption made of the input data is identical to the above, with the exception that this time I use 10% of the full dataset rather than jus the first 3 items

In [ ]:
# Import csv module to interact with the input data
import csv
import math
import sys

points_list = []
shortest_pt_distance = float('inf')
next_point = []
best_path = []
best_distance = float('inf')
input_data = "../input/a1-data/cities10per.csv"  # Enter filename of input data here.
output_data = "output.csv"  # Enter filename of output data here
starting_point = [0.0, 0.0, 0.0]  # Starting point declared here. Format: [0.0, x-coord, y-coord]
ending_point = [0.0, 0.0, 0.0]  # Ending point declared here. Format: [0.0, x-coord, y-coord]

with open(input_data, 'r') as csvfile:  # reading the csv file
    csvreader = csv.reader(csvfile)     # Creating a reader
    col_names = next(csvreader)         # Extract column names
    for point_STR in csvreader:         # Add data points into internal list
        point_FLT = []                  # Convert the data points from String to Float
        for i in point_STR:
            point_FLT.append(float(i))
        points_list.append(point_FLT)
        
def save_best_path():
    with open(output_data, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(best_path)
        
def find_dist(pt1, pt2):                # Function to find the distance between 2 points
    pt1 = int(pt1)                      # Calculation below assumes points are on flat surface
    pt2 = int(pt2)
    distance = math.sqrt(((points_list[pt1][1] - points_list[pt2][1])**2) + ((points_list[pt1][2] - points_list[pt2][2])**2))
    return distance

def find_dist_start(pt):                # Extra function used to account for starting point if provided
    pt = int(pt)                        # Calculation below assumes points are on flat surface
    distance = math.sqrt(((starting_point[1] - points_list[pt][1])**2) + ((starting_point[2] - points_list[pt][2])**2))
    return distance

def find_dist_end(pt):                  # Extra function used to account for ending point if provided
    pt = int(pt)                        # Calculation below assumes points are on flat surface
    distance = math.sqrt(((ending_point[1] - points_list[pt][1])**2) + ((ending_point[2] - points_list[pt][2])**2))
    return distance

These next lines create a new, copied list of points from the one generated above, with a key difference being that it is sorted by its x-coordinate values.

In [ ]:
points_list_sortedX = []

points_list_sortedX = points_list.copy()
points_list_sortedX.sort(key=lambda x: x[1])

The following function is responsible for finding the closest unused point to be next connected to.
<br>Instead of checking every point possible and finding the shortest distance among them, this function uses the sorted list generated above to narrow the number of points to be checked.
<br>
<br>This is done by creating a for loop that only checks the current points <b>neighbours</b> in the sorted list, breaking the for loop if a neighbour in either the forward or backward direction is out of range, and saving the valid points to a new list.
<br>After generating a list of possible points to connect to by x-coordinate range, this new list is evaluated in the same way for its y-coordinate value, further reducing the number of points to check for direct distance.
<br>To explain this process, it is first a reduction of possible points on a 2-d plane to a column that is my search range, and then further reducing the column to a box, with sides equal to my search range.
<br>
<br>Once the number of possible points has been reduced to those within the defined box, the function then uses brute force to find the closest point among them.
<br>Alternatively, if no points are withing the search range, the function recursively expands the search range until possible points are found.
<br>
<br>Time complexity: 8n + 8

In [ ]:
def find_closest_pt(curr_point, search_range):              # Recursive function to find closest point
    curr_x_index = points_list_sortedX.index(curr_point)    # Defines the index of current point on sorted list
    possible_points_1 = []
    possible_points_2 = []
    dist_holder = float('inf')
    index_holder = 0.0
    if len(points_list_sortedX) == 1:                                   # Checks if this is the last point is list
        return False                                                    # Returns False if yes
    if curr_x_index > 0:                                                # Checks if any points behind current point
        for element in points_list_sortedX[curr_x_index-1::-1]:         # If yes, iterate backwards to find points
            if abs(curr_point[1] - element[1]) < search_range:
                possible_points_1.append(element)                       # Saves points if x coords in range
            else:
                break                                                   # Ends loop if no more points in range
    for element in points_list_sortedX[curr_x_index + 1:]:              # Now iterate forwards to find points
        if abs(curr_point[1] - element[1]) < search_range:
            possible_points_1.append(element)                           # Saves points to list1 if x coords in range
        else:
            break                                                       # Ends loop if no more points in range
    if curr_point in possible_points_1:
        possible_points_1.remove(curr_point)
    if len(possible_points_1) > 0:                                      # While there are any possible points
        possible_points_1.sort(key=lambda x: x[2])                      # Sort points by Y coords
        for point2 in possible_points_1:
            if abs(curr_point[2] - point2[2]) < search_range:           # Check if points Y coords are also in range
                possible_points_2.append(point2)                        # Saves points to list2 if yes

        if len(possible_points_2) > 0:                                  # If there are still possible points
            for point3 in possible_points_2:                            # Check all point's distance for closest
                if find_dist(point3[0], curr_point[0]) < dist_holder:
                    dist_holder = find_dist(point3[0], curr_point[0])
                    index_holder = point3[0]
            return index_holder                                         # Returns index of closest point
    return find_closest_pt(curr_point, (search_range + 100)*2)          # Re-curves with bigger range if no points

Lastly, the plot_path function is what utilizes the above functions, taking the returned closest points and adding it to the final 'best_path' that is the answer.
<br>Additionally, I had added a simple progress tracker, printing a percentage completion so as to show that the process is still active.
<br>
<br>Time complexity: 6n + 7

In [ ]:
def plot_path():                                                        # Function that plots the best path
    global best_path                                                    # Initiates the global variables to record
    global best_distance                                                #   best path and distance
    points_list_sortedX.insert(0, starting_point)                       # Insert the starting point to points list
    best_path.append(points_list[int(find_closest_pt(starting_point, 100))])    # Tracks first point
    best_distance = find_dist_start(find_closest_pt(starting_point, 100))       # Measures starting distance
    points_list_sortedX.remove(starting_point)                          # Removes starting point from points list

    while len(points_list_sortedX) > 1:                                 # While there are unused points
        next_pt_index = find_closest_pt(best_path[-1], 100)             # Find the closest pt to the last added pt
        points_list_sortedX.remove(best_path[-1])                       # Remove last added pt from unused pts list
        best_path.append(points_list[int(next_pt_index)])               # Add the next point to path
        best_distance += find_dist(best_path[-2][0], best_path[-1][0])
        completion_percent = len(best_path)/len(points_list)
        print("Completion percent: ", completion_percent * 100, "%")

    best_path.append(points_list_sortedX[0])                            # Adds last point to end of the path
    best_distance += find_dist(best_path[-2][0], best_path[-1][0])      # Adds distance to last point
    best_distance += find_dist_end(best_path[-1][0])

In [ ]:
plot_path()
save_best_path()
global best_path                # Initiates the global variables to print progress into console
global best_distance
print("Best distance is: ", best_distance)
print("Using best path: ", best_path)

In [ ]:
from IPython.display import Image

![10perPathPlotted](10perPathPlotted.jpg)

<h3><u>Algorithm 2 review</u></h3>
<br>The first major difference if the improvement in time complexity. While the first algorithm had no hope of ever finishing even 1% of the data, the above can handily process and plot a path for 10%.
<br>Likewise, a downside is that this algorithm will absolutley not produce as optimal of paths as the first, with a potential for a path to leave points across the 2-D plane, which will take a large amount of distance to go back to. This can even be seen in the plotted image above, where there are some parts of the path that cross over large areas that had already been prviously visited.